In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import mpld3
from mpld3 import plugins
import numpy as np
import csv

mpld3.enable_notebook()

In [2]:
coord_list = []
species_list = []

with open('AllBirdsv4.csv', newline='') as f:
    reader = csv.reader(f)
    next(reader, None)  # skip the headers
    for row in reader:
        if isinstance(row[1], str) and row[1] not in species_list:
            species_list.append(row[1])
        if row[6].isdigit() and row[7].isdigit():
            coord_list.append((species_list.index(row[1]), int(row[6]), int(row[7])))

coord_test_list = []
with open('Test Birds Location.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        if row[1].isdigit() and row[2].isdigit():
            coord_test_list.append((int(row[1]), int(row[2])))

# check coordinates
for i in range(10):
    print(coord_list[i])

(0, 49, 63)
(0, 125, 133)
(0, 58, 76)
(0, 55, 125)
(0, 129, 123)
(0, 132, 121)
(0, 106, 107)
(0, 149, 115)
(0, 134, 118)
(0, 89, 131)


In [3]:
img = mpimg.imread('Lekagul Roadways 2018.bmp')
img = img[::-1] # reverse order for y-axis so that (0,0) is bottom-left

fig = plt.figure(figsize=(10,10))

map_plot = fig.add_subplot()
plt.imshow(img, origin='lower')
plt.title('Birds locations on Lekagul Roadways map', size=20)
plt.xlabel('x')
plt.ylabel('y')
cmap = plt.get_cmap('tab20')

# plot all birds
scatter_all_birds = plt.scatter([coord[1] for coord in coord_list], [coord[2] for coord in coord_list],
                                s=20, alpha=0.7, c=cmap([coord[0] for coord in coord_list]))

# plot the 15 "test birds"
scatter_test_birds = plt.scatter([coord[0] for coord in coord_test_list], [coord[1] for coord in coord_test_list], 
                                 s=300, c='red', marker='^')

# mouse hover on coordinates
labels = []
for i in range(len(coord_test_list)):
    labels.append("Test Bird {} ({}, {})".format(i+1, coord_test_list[i][0], coord_test_list[i][1]))
    
tooltip_test_birds = plugins.PointLabelTooltip(scatter_test_birds, labels=labels)
tooltip_all_birds = plugins.PointLabelTooltip(scatter_all_birds, labels=[species_list[c[0]] for c in coord_list])
plugins.connect(fig, tooltip_test_birds)
plugins.connect(fig, tooltip_all_birds)

mpld3.display(fig)